In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('raw_transactions.csv')
df.head()

,transaction_id,user_id,transaction_date,amount,payment_method,device,payment_status,failure_reason,retry_count,recovered_flag
0,1,1860,2024-05-24,1324.34,Debit Card,iOS,success,NaN,0,0
1,2,4772,2024-04-04,2237.72,Credit Card,iOS,success,NaN,0,0
2,3,4092,2024-02-02,3190.09,Debit Card,Web,success,NaN,0,0
3,4,1466,2024-02-25,940.45,Credit Card,Android,failed,network_error,0,0
4,5,5426,2024-05-04,3846.30,Credit Card,Web,success,NaN,0,0


In [3]:
df.shape

(30000, 10)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    30000 non-null  int64  
 1   user_id           30000 non-null  int64  
 2   transaction_date  30000 non-null  object 
 3   amount            30000 non-null  float64
 4   payment_method    30000 non-null  object 
 5   device            30000 non-null  object 
 6   payment_status    30000 non-null  object 
 7   failure_reason    4581 non-null   object 
 8   retry_count       30000 non-null  int64  
 9   recovered_flag    30000 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 2.3+ MB


fixing data types

In [6]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df['amount'] = df['amount'].astype(float)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    30000 non-null  int64         
 1   user_id           30000 non-null  int64         
 2   transaction_date  30000 non-null  datetime64[ns]
 3   amount            30000 non-null  float64       
 4   payment_method    30000 non-null  object        
 5   device            30000 non-null  object        
 6   payment_status    30000 non-null  object        
 7   failure_reason    4581 non-null   object        
 8   retry_count       30000 non-null  int64         
 9   recovered_flag    30000 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 2.3+ MB


In [5]:
df.isna().sum()

,0
transaction_id,0
user_id,0
transaction_date,0
amount,0
payment_method,0
device,0
payment_status,0
failure_reason,25419
retry_count,0
recovered_flag,0


handling null values

In [8]:
df['failure_reason'] = df['failure_reason'].fillna('no_failure')

In [9]:
df.isna().sum()

,0
transaction_id,0
user_id,0
transaction_date,0
amount,0
payment_method,0
device,0
payment_status,0
failure_reason,0
retry_count,0
recovered_flag,0


Feature Engineering

1.Recovery Status

In [10]:
df['recovery_status'] = np.where(
    (df['payment_status'] == 'failed') & (df['recovered_flag'] == 1),
    'recovered',
    np.where(df['payment_status'] == 'failed', 'lost', 'success')
)

In [13]:
df['recovery_status'].value_counts()

,count
recovery_status,
success,25419
lost,3045
recovered,1536


2.Revenue Impact

In [11]:
df['revenue_lost'] = np.where(
    df['recovery_status'] == 'lost',
    df['amount'],
    0
)

df['revenue_recovered'] = np.where(
    df['recovery_status'] == 'recovered',
    df['amount'],
    0
)

In [14]:
df[['revenue_lost', 'revenue_recovered']].sum()

,0
revenue_lost,7719977.17
revenue_recovered,3992232.53


3.Retry Bucket

In [12]:
df['retry_bucket'] = pd.cut(
    df['retry_count'],
    bins=[-1, 0, 1, 2, 3],
    labels=['no_retry', '1_retry', '2_retries', '3_retries']
)

In [15]:
df[df['payment_status'] == 'success']['retry_count'].value_counts()

,count
retry_count,
0,25419


In [16]:
df.to_csv('cleaned_transactions.csv', index=False)
